# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages, which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission if necessary. 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission. 

In addition to implementing code, there is a writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) that can be used to guide the writing process. Completing the code template and writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/481/view) for this project.

The [rubric](https://review.udacity.com/#!/rubrics/481/view) contains "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. The stand out suggestions are optional. If you decide to pursue the "stand out suggestions", you can include the code in this Ipython notebook and also discuss the results in the writeup file.


>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [ ]:
# Load pickled data
import pickle

training_file = './traffic-signs-data/train.p'
validation_file= './traffic-signs-data/valid.p'
testing_file = './traffic-signs-data/test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below. Use python, numpy and/or pandas methods to calculate the data summary rather than hard coding the results. For example, the [pandas shape method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shape.html) might be useful for calculating some of the summary results. 

### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [ ]:
### Replace each question mark with the appropriate value. 
### Use python, pandas or numpy methods rather than hard coding the results

# TODO: Number of training examples
n_train = X_train.shape[0]

# TODO: Number of testing examples.
n_test = X_test.shape[0]

# TODO: What's the shape of an traffic sign image?
image_shape = X_train[0].shape

# TODO: How many unique classes/labels there are in the dataset.
n_classes = len(set(y_train))

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

### Include an exploratory visualization of the dataset

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc.

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections.

In [ ]:
### Data exploration visualization code goes here.
### Feel free to use as many code cells as needed.
import matplotlib.pyplot as plt
import matplotlib.cm
import random
# Visualizations will be shown in the notebook.
%matplotlib inline

def display_random_images(x, y, x_cnt=5, y_cnt=2):
    plt.rcdefaults()
    
    fig, axes = plt.subplots(y_cnt, x_cnt, figsize=(12, 4))
    fig.subplots_adjust(hspace=0.5, wspace=0.05)
    for ax in axes.flat:
        idx = random.randint(0, len(x))
        if len(x[idx].shape) < 3 or x[idx].shape[2] == 1:
            img = x[idx].copy()
            ax.imshow(img.reshape(img.shape[0:2]), cmap=matplotlib.cm.gray)
        else:
            ax.imshow(x[idx])
        ax.set_title("class={}".format(y[idx]))
        ax.axis('off')
    
    plt.show()


def display_images(images, y_cnt, x_cnt, figsize=(2, 2), hspace=0.5, wspace=0.05):
    plt.rcdefaults()
    
    fig, axes = plt.subplots(y_cnt, x_cnt, figsize=figsize)
    fig.subplots_adjust(hspace=hspace, wspace=wspace)
    idx = 0
    if (x_cnt, y_cnt) == (1, 1):
        axes = [axes]
    else:
        axes = axes.flat
    for ax in axes:
        if idx >= len(images):
            break
        image = images[idx]
        if len(image.shape) < 3 or image.shape[2] == 1:
            img = image.copy()
            ax.imshow(img.reshape(img.shape[0:2]), cmap=matplotlib.cm.gray)
        else:
            ax.imshow(image)
        idx += 1
        #ax.set_title("class={}".format(y[idx]))
        ax.axis('off')
    
    plt.show()
    
display_random_images(X_train, y_train)

In [ ]:
def count_by_class(y_train):
    result = {}
    for cls in y_train:
        if cls not in result:
            result[cls] = 1
        else:
            result[cls] += 1
    return result

def group_by_class(x_train, y_train):
    """Returns {cls: [idx of items in x_train with class cls], ...}
    """
    result = {}
    for idx in range(len(x_train)):
        cls = y_train[idx]
        result.setdefault(cls, []).append(idx)
    return result

def plot_counts_by_class(y_train, title):
    counts = count_by_class(y_train)
    sorted_keys = sorted(counts.keys())
    sorted_values = [counts[k] for k in sorted_keys]

    plt.rcdefaults()
    fig, ax = plt.subplots(figsize=(12, 5))
    ax.bar(sorted_keys, sorted_values, width=0.75)
    ax.set_xticks(sorted_keys)
    ax.set_xlabel('Class')
    ax.set_title(title)
    plt.show()

plot_counts_by_class(y_train, 'Training examples by class')
#plot_counts_by_class(y_test, 'Test examples by class')

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

There are various aspects to consider when thinking about this problem:

- Neural network architecture
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

**NOTE:** The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play!

### Pre-process the Data Set (normalization, grayscale, etc.)

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project.

In [ ]:
from skimage.util import dtype
import numpy as np

def _prepare_colorarray(arr):
    """Check the shape of the array and convert it to
    floating point representation.
    """
    arr = np.asanyarray(arr)

    if arr.ndim not in [3, 4] or arr.shape[-1] != 3:
        msg = ("the input array must be have a shape == (.., ..,[ ..,] 3)), " +
               "got (" + (", ".join(map(str, arr.shape))) + ")")
        raise ValueError(msg)

    return dtype.img_as_float(arr)


def rgb2hsv(rgb):
    """RGB to HSV color space conversion.
    Parameters
    ----------
    rgb : array_like
        The image in RGB format, in a 3-D array of shape ``(.., .., 3)``.
    Returns
    -------
    out : ndarray
        The image in HSV format, in a 3-D array of shape ``(.., .., 3)``.
    Raises
    ------
    ValueError
        If `rgb` is not a 3-D array of shape ``(.., .., 3)``.
    Notes
    -----
    Conversion between RGB and HSV color spaces results in some loss of
    precision, due to integer arithmetic and rounding [1]_.
    References
    ----------
    .. [1] http://en.wikipedia.org/wiki/HSL_and_HSV
    Examples
    --------
    >>> from skimage import color
    >>> from skimage import data
    >>> img = data.astronaut()
    >>> img_hsv = color.rgb2hsv(img)
    """
    arr = _prepare_colorarray(rgb)
    out = np.empty_like(arr)

    # -- V channel
    out_v = arr.max(-1)

    # -- S channel
    delta = arr.ptp(-1)
    # Ignore warning for zero divided by zero
    old_settings = np.seterr(invalid='ignore')
    out_s = delta / out_v
    out_s[delta == 0.] = 0.

    # -- H channel
    # red is max
    idx = (arr[:, :, 0] == out_v)
    out[idx, 0] = (arr[idx, 1] - arr[idx, 2]) / delta[idx]

    # green is max
    idx = (arr[:, :, 1] == out_v)
    out[idx, 0] = 2. + (arr[idx, 2] - arr[idx, 0]) / delta[idx]

    # blue is max
    idx = (arr[:, :, 2] == out_v)
    out[idx, 0] = 4. + (arr[idx, 0] - arr[idx, 1]) / delta[idx]
    out_h = (out[:, :, 0] / 6.) % 1.
    out_h[delta == 0.] = 0.

    np.seterr(**old_settings)

    # -- output
    out[:, :, 0] = out_h
    out[:, :, 1] = out_s
    out[:, :, 2] = out_v

    # remove NaN
    out[np.isnan(out)] = 0

    return out


def _convert(matrix, arr):
    """Do the color space conversion.
    Parameters
    ----------
    matrix : array_like
        The 3x3 matrix to use.
    arr : array_like
        The input array.
    Returns
    -------
    out : ndarray, dtype=float
        The converted array.
    """
    arr = _prepare_colorarray(arr)

    return np.dot(arr, matrix.T.copy())


yuv_from_rgb = np.array([[ 0.299     ,  0.587     ,  0.114      ],
                         [-0.14714119, -0.28886916,  0.43601035 ],
                         [ 0.61497538, -0.51496512, -0.10001026 ]])

def rgb2yuv(rgb):
    """RGB to YUV color space conversion.
    Parameters
    ----------
    rgb : array_like
        The image in RGB format, in a 3- or 4-D array of shape
        ``(M, N, [P,] 3)``.
    Returns
    -------
    out : ndarray
        The image in YUV format, in a 3- or 4-D array of shape
        ``(M, N, [P,] 3)``.
    Raises
    ------
    ValueError
        If `rgb` is not a 3- or 4-D array of shape ``(M, N, [P,] 3)``.
    Notes
    -----
    Y is between 0 and 1.  Use YCbCr instead of YUV for the color space which
    is commonly used by video codecs (where Y ranges from 16 to 235)
    References
    ----------
    .. [1] https://en.wikipedia.org/wiki/YUV
    """
    return _convert(yuv_from_rgb, rgb)


from skimage.color import rgb2grey


def greyscale(data):
    output_shape = list(data.shape)
    output_shape[3] = 1
    greyscaled = (rgb2grey(data) / 255.0).astype(np.float32)
    return greyscaled.reshape(output_shape)


def to_y_channel(data):
    result = 0.299 * data[:, :, :, 0] + 0.587 * data[:, :, :, 1] + 0.114 * data[:, :, :, 2]
    result = (result / 255.).astype(np.float32)
    return result.reshape(result.shape + (1,))


import skimage.exposure


def equalize_hist(in_image):
    return skimage.exposure.equalize_adapthist(in_image.reshape(in_image.shape[0:2])).reshape(in_image.shape)


def equalize_hist_all(images):
    total = images.shape[0]
    for i in range(total):
        if total > 500 and i % 500 == 0:
            print("Processing {} out of {}".format(i, total))
        images[i] = equalize_hist(images[i])
        

def normalize_input_1(data):
    result = greyscale(data)
    equalize_hist_all(result)
    return result


def normalize_input_2(data):
    result = to_y_channel(data)
    equalize_hist_all(result)
    return result

# Select version
normalize_input = normalize_input_1

## Example images after preprocessing

In [ ]:
indices = np.random.randint(0, len(X_train), 5)
example_images = X_train[indices]
preprocessed_images_1 = normalize_input_1(example_images)
preprocessed_images_2 = normalize_input_2(example_images)
display_images(list(example_images) + list(preprocessed_images_1) + list(preprocessed_images_2),
               3, 5, figsize=(6, 5))

In [ ]:
### Preprocess the data here. Preprocessing steps could include normalization, converting to grayscale, etc.
### Feel free to use as many code cells as needed.

import skimage.util
import skimage.transform
import numpy as np


def add_salt_pepper(src_image, amount, salt_vs_pepper):
    def rand_dim(dim, count):
        if dim == 1:
            return 0
        else:
            return np.random.randint(0, dim-1, count)

    result = src_image.copy()
    salt_count = np.ceil(amount * src_image.size * salt_vs_pepper)
    pepper_count = np.ceil(amount * src_image.size * (1.0 - salt_vs_pepper))
    salt_coords = [rand_dim(dim, int(salt_count)) for dim in src_image.shape]
    pepper_coords = [rand_dim(dim, int(pepper_count)) for dim in src_image.shape]
    result[salt_coords] = 1
    result[pepper_coords] = 0
    return result


def extend_dataset_to_avg(x, y):
    groups_by_class = group_by_class(x, y)
    avg_counts = int(sum((len(lst) for lst in groups_by_class.values())) / len(groups_by_class))
    
    print("Got avg_counts={}".format(avg_counts))

    manglers = [
        lambda img: skimage.util.random_noise(img, 'gaussian', mean=0.1, var=0.002),
        #lambda img: skimage.util.random_noise(img, 's&p', amount=0.008, salt_vs_pepper=0.5),
        lambda img: add_salt_pepper(img, amount=0.008, salt_vs_pepper=0.5)
    ]

    addon_x = []
    addon_y = []
    for cls, indices in groups_by_class.items():
        if len(indices) >= avg_counts:
            continue

        for img_idx in range(len(indices), avg_counts+1):
            rand_image_idx = indices[np.random.randint(0, len(indices))]
            src_image = x[rand_image_idx]
            src_cls = y[rand_image_idx]
            mangler = manglers[np.random.randint(0, len(manglers))]
            addon_x.append(mangler(src_image))
            addon_y.append(src_cls)

    return np.stack(addon_x), np.stack(addon_y)


def extend_dataset(x, y):

    manglers = [
        #lambda img: skimage.util.random_noise(img, 'gaussian', mean=0.1, var=0.002),
        #lambda img: add_salt_pepper(img, amount=0.008, salt_vs_pepper=0.5),
        #lambda img: skimage.transform.rotate(img, np.random.randint(-15, 15)),
        #lambda img: skimage.transform.warp(img, skimage.transform.AffineTransform(
        #    scale=[1.0 + (np.random.random() * 0.2 - 0.1)]*2)),
        lambda img: skimage.transform.warp(img, skimage.transform.AffineTransform(translation=np.random.randint(-2, 2))),
        # Other realistic perturbations would probably also increase robustness such as other affine transformations,
        # brightness, contrast and blur
    ]

    addon_x = []
    addon_y = []
    for img_idx in range(len(x)):
        src_image = x[img_idx]
        src_cls = y[img_idx]
        for mangler in manglers:
            addon_x.append(mangler(src_image))
            addon_y.append(src_cls)

    return np.stack(addon_x), np.stack(addon_y)


EXTEND_DATASET = False


if EXTEND_DATASET:
    X_train_addon, y_train_addon = extend_dataset(X_train, y_train)

    #display_random_images(X_train_addon, y_train_addon)

    X_train_extended = np.concatenate((X_train, X_train_addon))
    y_train_extended = np.concatenate((y_train, y_train_addon))
else:
    X_train_extended = X_train
    y_train_extended = y_train


X_train_extended = normalize_input(X_train_extended)
image_shape = X_train_extended[0].shape
X_test = normalize_input(X_test)
X_valid = normalize_input(X_valid)

display_random_images(X_train_extended, y_train)
    
#plot_counts_by_class(y_train_extended, 'Training examples by class - extended')

In [ ]:
training_file = './traffic-signs-data/train_grayscale_equalized.p'
validation_file= './traffic-signs-data/valid_grayscale_equalized.p'
testing_file = './traffic-signs-data/test_grayscale_equalized.p'

In [ ]:
# Storing preprocessed images

import pickle

with open(training_file, mode='wb') as f:
    pickle.dump({'features': X_train_extended, 'labels': y_train_extended}, f)
with open(testing_file, mode='wb') as f:
    pickle.dump({'features': X_valid, 'labels': y_valid}, f)
with open(validation_file, mode='wb') as f:
    pickle.dump({'features': X_test, 'labels': y_test}, f)

In [ ]:
# Loading preprocessed images

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)

X_train_extended, y_train_extended = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

image_shape = X_train_extended[0].shape

In [ ]:
from sklearn.utils import shuffle

X_train_extended, y_train_extended = shuffle(X_train_extended, y_train_extended)

### Model Architecture

In [ ]:
### Define your architecture here.

import tensorflow as tf


def get_shape(input_shape, output_shape, strides):
    return input_shape + strides - output_shape*strides


class ChainBuilder(object):
    def __init__(self, mean=None, stddev=None):
        self._output = None
        self._input = None
        self._output_shape = None
        self._mean = mean
        self._stddev = stddev

    def input(self, input_data, input_shape):
        self._input = input_data
        self._output = input_data
        self._output_shape = input_shape
        return self

    def conv2d(self, filter_shape, strides, name):
        input_shape = self._output_shape
    
        if len(input_shape) < 3:
            input_depth = 1
        else:
            input_depth = input_shape[2]
            
        with tf.variable_scope(name):
            weights = tf.get_variable('weights',
                                      shape=(filter_shape[0], filter_shape[1], input_depth, filter_shape[2]))
        
            biases = tf.get_variable('biases',
                                     initializer=tf.zeros_initializer(),
                                     shape=filter_shape[2])
        
            self._output = tf.nn.conv2d(self._output, weights, strides=[1]+strides+[1], padding='VALID') + biases
            self._output_shape = self._output.shape.as_list()[1:]
        print("conv2d result: output_shape={}, output={}".format(self._output_shape, self._output))
        return self
    
    @property
    def output(self):
        return self._output
    
    @property
    def output_shape(self):
        return self._output_shape
    
    def relu(self):
        self._output = tf.nn.relu(self._output)
        return self
    
    def max_pool(self, kernel, strides):
        assert len(self._output_shape) == 3
        self._output = tf.nn.max_pool(self._output,
                                      ksize=[1, kernel[0], kernel[1], 1],
                                      strides=[1]+strides+[1],
                                      padding='SAME')
        self._output_shape = self._output.shape.as_list()[1:]
        #print("max_pool result: output_shape={}, output={}".format(self._output_shape, self._output))
        return self
    
    def flatten(self):
        assert len(self._output_shape) == 3
        self._output = tf.contrib.layers.flatten(self._output)
        self._output_shape = [self._output_shape[0] * self._output_shape[1] * self._output_shape[2]]
        #print("flatten result: output_shape={}, output={}".format(self._output_shape, self._output))
        return self
    
    def concat(self, what):
        self._output = tf.concat([self._output] + what, 1)
        self._output_shape = [self._output.shape.as_list()[1]]
        return self
    
    def fc(self, output_n, name):
        assert len(self._output_shape) == 1
        #print("Creating fc({}, {}, {})".format(self._output_shape, output_n, self._output))

        with tf.variable_scope(name):
            weights = tf.get_variable('weights',
                                     shape=(self._output_shape[0], output_n))
            biases = tf.get_variable('biases',
                                     initializer=tf.zeros_initializer(),
                                     shape=(output_n,))
        
        fc = tf.matmul(self._output, weights) + biases
        self._output = fc
        self._output_shape = [output_n]
        return self

In [ ]:
# Standard LeNet architecture produces 0.753 validation accuracy and about 0.2 accuracy on web dataset
def create_model_lenet(input_data, n_classes):
    with tf.variable_scope('lenet1', initializer=tf.truncated_normal_initializer(mean=0, stddev=0.1)):
        builder = ChainBuilder()
        builder.input(input_data, input_data[0].shape.as_list()[0:3]).\
            conv2d([5, 5, 6], [1, 1]).relu().\
            max_pool([2, 2], [2, 2]).\
            conv2d([5, 5, 16], [1, 1]).relu().\
            max_pool([2, 2], [2, 2]).\
            flatten().\
            fc(400).relu().\
            fc(100).relu().\
            fc(n_classes)
    return builder.output

# Multiscale
def create_model(input_data, n_classes):
    with tf.variable_scope('ms10', initializer=tf.truncated_normal_initializer(mean=0, stddev=0.1)):
        builder1 = ChainBuilder()
        builder1.input(input_data, input_data[0].shape.as_list()[0:3]).\
            conv2d([5, 5, 32], [1, 1], 'conv_1').relu().\
            max_pool([2, 2], [2, 2])
        builder2 = ChainBuilder()
        builder2.input(builder1.output, builder1.output_shape).\
            conv2d([5, 5, 128], [1, 1], 'conv_2').relu().\
            max_pool([2, 2], [2, 2]).\
            flatten().\
            concat([builder1.max_pool([2, 2], [2, 2]).flatten().output]).\
            fc(400, 'fc_1').relu().\
            fc(n_classes, 'fc_2')
    return builder2.output

### Train, Validate and Test the Model

A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the training set but low accuracy on the validation set implies overfitting.

In [ ]:
### Train your model here.
### Calculate and report the accuracy on the training and validation set.
### Once a final model architecture is selected, 
### the accuracy on the test set should be calculated and reported as well.
### Feel free to use as many code cells as needed.

EPOCHS = 10
BATCH_SIZE = 128
learning_rate = 0.001

In [ ]:
input_x = tf.placeholder(tf.float32, [None] + list(image_shape))
input_y = tf.placeholder(tf.int32, (None, ))

output_one_hot_y = tf.one_hot(input_y, n_classes)
logits = create_model(input_x, n_classes)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=output_one_hot_y)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_operation = optimizer.minimize(loss_operation)
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(output_one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

def evaluate(X_data, y_data, batch_size):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, batch_size):
        batch_x, batch_y = X_data[offset:offset+batch_size], y_data[offset:offset+batch_size]
        accuracy = sess.run(accuracy_operation, feed_dict={input_x: batch_x, input_y: batch_y})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

In [ ]:
def train_the_model(X_train, y_train, X_valid, y_valid, batch_size, epochs):
    saver = tf.train.Saver()
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        num_examples = len(X_train)
        
        print("Training...")
        print()
        for i in range(epochs):
            X_train, y_train = shuffle(X_train, y_train)
            for offset in range(0, num_examples, batch_size):
                end = offset + batch_size
                batch_x, batch_y = X_train[offset:end], y_train[offset:end]
                sess.run(training_operation, feed_dict={input_x: batch_x, input_y: batch_y})
                
            validation_accuracy = evaluate(X_valid, y_valid, batch_size)
            print("EPOCH {} ...".format(i+1))
            print("Validation Accuracy = {:.3f}".format(validation_accuracy))
            print()
            
        saver.save(sess, './lenet')
        print("Model saved")

In [ ]:
train_the_model(X_train_extended, y_train_extended, X_valid, y_valid, BATCH_SIZE, EPOCHS)

---

## Step 3: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.

import skimage.io
import skimage.transform
import os

images_dir = './web_images'
images = []
classes = []
for cls_id in os.listdir(images_dir):
    cls_dir = os.path.join(images_dir, cls_id)
    if os.path.isdir(cls_dir):
        for fname in os.listdir(cls_dir):
            if fname.endswith(('.jpg', '.png')):
                images.append(skimage.io.imread(os.path.join(cls_dir, fname)))
                classes.append(int(cls_id))
        

resized = [
    skimage.transform.resize(img, (image_shape[0], image_shape[1]))
    for img in images
]

display_images(resized, 1, len(resized))

### Predict the Sign Type for Each Image

In [ ]:
### Run the predictions here and use the model to output the prediction for each image.
### Make sure to pre-process the images with the same pre-processing pipeline used earlier.
### Feel free to use as many code cells as needed.

X_web = normalize_input(np.asarray(resized))
y_web = np.asarray(classes)

saver = tf.train.Saver()

# Launch the graph
with tf.Session() as sess:
    saver.restore(sess, './lenet')
    
    sess = tf.get_default_session()
    result = sess.run(tf.argmax(logits, 1), feed_dict={input_x: X_web, input_y: y_web})

    print("Result   = {}".format(result))
    print("Expected = {}".format(y_web))
    print()

### Analyze Performance

In [ ]:
### Calculate the accuracy for these 5 new images. 
### For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate on these new images.

with tf.Session() as sess:
    saver.restore(sess, './lenet')
    validation_accuracy = evaluate(X_web, y_web, BATCH_SIZE)
    print("Validation Accuracy = {:.3f}".format(validation_accuracy))

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. 

The example below demonstrates how tf.nn.top_k can be used to find the top k predictions for each image.

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example. The values in the array represent predictions. The array contains softmax probabilities for five candidate images with six possible classes. `tk.nn.top_k` is used to choose the three classes with the highest probability:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

In [ ]:
### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. 
### Feel free to use as many code cells as needed.

---

## Step 4: Visualize the Neural Network's State with Test Images

 This Section is not required to complete but acts as an additional excersise for understaning the output of a neural network's weights. While neural networks can be a great learning device they are often referred to as a black box. We can understand what the weights of a neural network look like better by plotting their feature maps. After successfully training your neural network you can see what it's feature maps look like by plotting the output of the network's weight layers in response to a test stimuli image. From these plotted feature maps, it's possible to see what characteristics of an image the network finds interesting. For a sign, maybe the inner network feature maps react with high activation to the sign's boundary outline or to the contrast in the sign's painted symbol.

 Provided for you below is the function code that allows you to get the visualization output of any tensorflow weight layer you want. The inputs to the function should be a stimuli image, one used during training or a new one you provided, and then the tensorflow variable name that represents the layer's state during the training process, for instance if you wanted to see what the [LeNet lab's](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) feature maps looked like for it's second convolutional layer you could enter conv2 as the tf_activation variable.

For an example of what feature map outputs look like, check out NVIDIA's results in their paper [End-to-End Deep Learning for Self-Driving Cars](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/) in the section Visualization of internal CNN State. NVIDIA was able to show that their network's inner weights had high activations to road boundary lines by comparing feature maps from an image with a clear path to one without. Try experimenting with a similar test to show that your trained network's weights are looking for interesting features, whether it's looking at differences in feature maps from images with or without a sign, or even what feature maps look like in a trained network vs a completely untrained one on the same sign image.

<figure>
 <img src="visualize_cnn.png" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above)</p> 
 </figcaption>
</figure>
 <p></p> 


In [ ]:
### Visualize your network's feature maps here.
### Feel free to use as many code cells as needed.

# image_input: the test image being fed into the network to produce the feature maps
# tf_activation: should be a tf variable name used during your training procedure that represents the calculated state of a specific weight layer
# activation_min/max: can be used to view the activation contrast in more detail, by default matplot sets min and max to the actual min and max values of the output
# plt_num: used to plot out multiple different weight feature map sets on the same block, just extend the plt number for each new feature map entry

def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1 ,plt_num=1):
    # Here make sure to preprocess your image_input in a way your network expects
    # with size, normalization, ect if needed
    # image_input =
    # Note: x should be the same name as your network's tensorflow data placeholder variable
    # If you get an error tf_activation is not defined it maybe having trouble accessing the variable from inside a function
    activation = tf_activation.eval(session=sess,feed_dict={x : image_input})
    featuremaps = activation.shape[3]
    plt.figure(plt_num, figsize=(15,15))
    for featuremap in range(featuremaps):
        plt.subplot(6,8, featuremap+1) # sets the number of feature maps to show on each row and column
        plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max, cmap="gray")
        elif activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
        elif activation_min !=-1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
        else:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", cmap="gray")

### Question 9

Discuss how you used the visual output of your trained network's feature maps to show that it had learned to look for interesting characteristics in traffic sign images


**Answer:**

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

### Project Writeup

Once you have completed the code implementation, document your results in a project writeup using this [template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) as a guide. The writeup can be in a markdown or pdf file. 